### 같은 평면상에 있는 error가 추가된 points

 - 이전 단계에서 수행한 points를 다시 load
 
 - 또한 기존의 STL data로 만들어진 point로부터 최근접 point를 찾기 위해서 다시 load 

In [17]:
import numpy as np

# 다시 읽어드리기

each_point_data = np.load('data/femur_each_point_data.npy')
points40_with_error_on_plane = np.load('data/points40_with_error_on_plane.npy')

###### 최근접 점 찾기

 - STL points data로부터 최근접 point를 찾기
 
 - kd tree 방법을 사용
 
 - ref
 
 > https://stackoverflow.com/questions/48126771/nearest-neighbour-search-kdtree

In [18]:
import math
from sklearn.neighbors import KDTree

In [19]:
# 근접한 점을 찾기위한 데이터 그룹 A로 tree 생성
tree = KDTree(each_point_data)

# 근접한 점을 찾기위한 대상 데이터 그룹 B를 tree.query에 입력 값으로 사용
nearest_dist, nearest_ind = tree.query(points40_with_error_on_plane, k=2)  # k=2 nearest neighbors where k1 = identity

# 대상 B로부터 가장 가까운 A 내의 점과의 거리 
print(nearest_dist[:, 1])

# 대상 B로부터 가장 가까운 A 그룹 내의 점의 인덱스 
print(nearest_ind[:, 1])

[2.71071472 2.93015618 2.66547745 1.09707971 1.83525775 0.81447184
 2.88230861 1.08016363 1.30728635 0.58131063 1.33546675 2.359163
 1.29720958 4.69117388 1.56041168 1.00159714 0.77455454 1.26429537
 0.97432329 1.64319471 3.09558545 0.78734078 1.30839626 0.91009284
 0.90700055 0.66877921 2.37557565 1.78203921 2.33935502 1.24234993
 0.43008194 1.71094157 0.70563205 2.29714374 2.56194244 0.39556087
 0.81782344 1.3649553  0.86587371 0.87310908]
[ 2812 21974 31866 16671 46593 21706  8932  6232  8627 16174 24663 16936
 12020  4618 10087 28886 55534 33746 42368 37740 54240 24983 51204 22384
 54137 26700 34479 32144 28480 55328 93821 57735 51747 92591 77346 76072
 92157 93569 74890 47731]


In [20]:
nearest_index = nearest_ind[:, 1]

nearest_points40_with_error_on_plane = []

for i in range(len(nearest_index)):
    nearest_points40_with_error_on_plane.append(each_point_data[nearest_index[i]])

 - 위에서 계산된 최근접 점이 정말 맞는지 distance를 계산하여 확인

In [21]:
nearest_distance = nearest_dist[:, 1]

test_point = points40_with_error_on_plane[0] - each_point_data[nearest_index[0]]

test_dist = math.sqrt(pow(test_point[0], 2) + pow(test_point[1], 2) + pow(test_point[2], 2))

if(nearest_distance[0] == test_dist):
    print('true!')

print(nearest_distance[0])
print(test_dist)

true!
2.7107147196201784
2.7107147196201784


In [22]:
# module
def get_closet_points(ref_points, trg_points, with_trg_index=False):
    """
    input data structure
    ref_points = [[xr0, yr0, zr0], ... [xrn, yrn, zrn]]
    trg_points = [[xt0, yt0, zt0], ... [xtm, ytm, ztm]]
    
    output data structure (closet points in trg_points from ref_points)
    closet_points = [[xc0, yc0, zc0], ... [xcn, ycn, zcn]]
    """
    tree = KDTree(trg_points)
    nearest_dist, nearest_ind = tree.query(ref_points, k=2)
    
    nearest_index = nearest_ind[:,1]
    closet_points = []
    
    for i in range(len(nearest_index)):
        closet_points.append(trg_points[nearest_index[i]])
        
    if (with_trg_index):
        return closet_points

In [23]:
# test! with func.
closet_points = get_closet_points(points40_with_error_on_plane, each_point_data)
test_point = points40_with_error_on_plane[0] - closet_points[0]
test_dist = math.sqrt(pow(test_point[0], 2) + pow(test_point[1], 2) + pow(test_point[2], 2))
if(nearest_distance[0] == test_dist):
    print('true!')

true!


###### npy로 저장해두기

In [24]:
import numpy as np

np.save('data/nearest_points40_with_error_on_plane', np.array(nearest_points40_with_error_on_plane))